Here's an outline for a Jupyter notebook that creates a Selenium workflow:

## Launches Chrome Debugger, Navigates and Logs Into Uber Driver Portal
* **Import Required Libraries**
* **Set Up Chrome Debugger**
* **Initialize WebDriver**
* **Navigate to Webpage**
* **Login In to UberDriver**



After the Cell Below Runs and User puts in 2 factor verification, it will put us on the https://drivers.uber.com/earnings/activities and we will be ready to run Step 2 in the work flow


Create Notebook

In [35]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import time


def launch_chrome_debugger(chrome_path, debug_port, user_data_dir):
    """Launch Chrome in debugging mode if not already running."""
    command = f'"{chrome_path}" --remote-debugging-port={debug_port} --user-data-dir="{user_data_dir}"'
    subprocess.Popen(command, shell=True)
    print(f"✅ Chrome launched in debugger mode on port {debug_port}")
    time.sleep(3)  # Give Chrome time to start


def attach_selenium_to_chrome(debug_port):
    """Attach Selenium WebDriver to the running Chrome instance."""
    chrome_options = Options()
    chrome_options.add_experimental_option("debuggerAddress", f"localhost:{debug_port}")
    driver = webdriver.Chrome(options=chrome_options)
    return driver


def login_uber_driver(driver, phone_number):
    """Automates Uber Driver login process if not already logged in."""
    driver.get("https://www.uber.com/")

    try:
        # Check if the 'Log in' button exists
        log_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(text(), 'Log in')]"))
        )
        print("🔄 Login page detected. Proceeding with login...")
        
        # Click 'Log in' button
        log_button.click()
        print("✅ Clicked 'Log in' button.")

        # Click 'Sign in to drive & deliver'
        signin_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//h2[contains(text(), 'Sign in to drive')]"))
        )
        signin_button.click()
        print("✅ Clicked 'Sign in to drive & deliver'.")

        # Enter phone number or email
        input_field = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "PHONE_NUMBER_or_EMAIL_ADDRESS"))
        )
        input_field.click()
        input_field.send_keys(phone_number)
        print(f"✅ Entered phone number: {phone_number}")

        # Click 'Continue' button
        forward_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "forward-button"))
        )
        forward_button.click()
        print("✅ Clicked 'Continue' button.")

        return True

    except Exception as e:
        print("⚠️ Log in button not found. Assuming user is already signed in.")
        driver.get("https://drivers.uber.com/earnings/activities")
        print("✅ Redirected to earnings page.")
        return False  # No login needed



# === Main Execution ===
if __name__ == "__main__":
    # Define Chrome paths and debugging settings
    CHROME_PATH = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
    DEBUG_PORT = "9222"
    USER_DATA_DIR = r"C:\selenium\ChromeProfile"
    PHONE_NUMBER = "5039010869"

    # Start Chrome in debug mode
    launch_chrome_debugger(CHROME_PATH, DEBUG_PORT, USER_DATA_DIR)

    # Attach Selenium to Chrome
    driver = attach_selenium_to_chrome(DEBUG_PORT)

    # Perform Uber login
    success = login_uber_driver(driver, PHONE_NUMBER)
    if success:
        print("✅ Uber Driver login successful!")
    else:
        print("❌ Uber Driver login failed. User likely already signed in.")

    

✅ Chrome launched in debugger mode on port 9222
🔄 Login page detected. Proceeding with login...
⚠️ Log in button not found. Assuming user is already signed in.
✅ Redirected to earnings page.
❌ Uber Driver login failed. User likely already signed in.


## Select Date Range 
###  Enter The Date at which point the subsequent Monday will be selected with the range ending on the following Sunday:
target_month = "January"
target_year = "2025"
target_day = "1"  # Only the day number
### Loads All Trips


In [36]:


# Ensure we are on the correct page
driver.get("https://drivers.uber.com/earnings/activities")

# Wait for the date input field to be clickable and click it
try:
    date_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@aria-label='Select a date range.']"))
    )
    date_input.click()
    print("✅ Clicked the date range input successfully.")
except Exception as e:
    print("❌ Error: Could not click the date range input.", e)

def select_month(driver, month_name):
    """Selects the specified month from the dropdown."""
    try:
        month_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Month')]"))
        )
        month_button.click()
        print("✅ Opened month dropdown.")

        month_list = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, f"//li[contains(text(), '{month_name}')]"))
        )
        driver.execute_script("arguments[0].click();", month_list)
        print(f"✅ Selected month: {month_name}.")
    except Exception as e:
        print(f"❌ Error selecting month: {month_name}.", e)

def select_year(driver, year):
    """Selects the specified year from the dropdown."""
    try:
        year_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Year')]"))
        )
        year_button.click()
        print("✅ Opened year dropdown.")

        year_option = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, f"//li[text()='{year}']"))
        )
        driver.execute_script("arguments[0].click();", year_option)
        print(f"✅ Selected year: {year}.")
    except Exception as e:
        print(f"❌ Error selecting year: {year}.", e)

def select_day(driver, target_day):
    """
    Selects a day from the currently displayed calendar.
    Example: target_day = "3"
    """
    try:
        day_xpath = f"//div[@role='gridcell' and .//div[normalize-space(text())='{target_day}']]"
        # Wait for the day element to be clickable (which implies it's present)
        selected_day = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, day_xpath))
        )
        selected_day.click()
        print(f"✅ Selected day: {target_day}.")
        return selected_day  # Returning the element in case it's needed later
    except Exception as e:
        print(f"❌ Error selecting day: {target_day}.", e)
        return None

def click_load_more(driver):
    count = 0
    while True:
        try:
            # Use a short timeout to check for the button
            load_more_button = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//button[@data-baseweb='button' and contains(text(),'Load More')]")
                )
            )
            load_more_button.click()
            count += 1
            print(f"✅ Clicked Load More button ({count} time{'s' if count > 1 else ''}).")
            time.sleep(2)  # Allow time for new trips to load
        except TimeoutException:
            print(f"No more 'Load More' button available. Total clicks: {count}")
            break
        except Exception as e:
            print("An error occurred while clicking Load More:", e)
            break
    return count


# Enter the target date details
target_month = "January"
target_year = "2025"
target_day = "1"  # Only the day number

select_month(driver, target_month)
select_year(driver, target_year)
selected_day = select_day(driver, target_day)
click_load_more(driver)





✅ Clicked the date range input successfully.
✅ Opened month dropdown.
✅ Selected month: January.
✅ Opened year dropdown.
✅ Selected year: 2025.
✅ Selected day: 1.
✅ Clicked Load More button (1 time).
✅ Clicked Load More button (2 times).
✅ Clicked Load More button (3 times).
✅ Clicked Load More button (4 times).
✅ Clicked Load More button (5 times).
No more 'Load More' button available. Total clicks: 5


5

## This is Scraping one individual Trip Details Page (Spcify one of many Trip Details Page to Test the Scrape).  This will be applied to a loop later on.

Pulls Text Based on 5 different Classes on the Trip Details Page

In [151]:

chrome_options = Options()
chrome_options.debugger_address = "localhost:9222"

# Attach Selenium to existing Chrome session
driver = webdriver.Chrome(options=chrome_options)

# Navigate to the Uber trip details page
driver.get("https://drivers.uber.com/earnings/trips/fc9037e5-e74c-46bb-8696-fdf0680d19c7")

# Wait for the button to be present and clickable
fare_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[@role='button' and .//p[text()='Fare']]"))
)

# Scrape the Details Page Based on the 4 different clases of the elements
xpath = (
    "//p[@class='_css-eXlLWF']"
    " | //span[@class='_css-jXJAeO']"
    " | //div[contains(@class, '_css-dTqljZ')]"
    " | //div[contains(@class, '_css-cFyYHT')]"
    " | //div[contains(@class, '_css-XloEJ')]"
    " | //p[@data-baseweb='typo-paragraphmedium' and contains(@class, '_css-dTqljZ')]"
)
elements = driver.find_elements(By.XPATH, xpath)
collected_values = [el.text for el in elements]

print(collected_values)




['10 min 49 sec', '2.90 mi', 'SE Lambert St, Portland, OR 97202-6102, US', 'SE Clinton St, Portland, OR 97202, US', '3 points earned', '$7.00 tip included', 'Fare', '$9.46', 'Tip', '$7.00', 'Total customer fare', '$19.10', 'Tip', '$7.00', 'Government taxes, third-party fees, and regulatory-related charges', '', '-$0.76', 'Estimated commercial auto insurance and operational expenses', '-$2.76', 'Your earnings', '-$16.46', 'Uber Service Fee', '$6.12', '', '', '', '']


## Activies Page Loop (Scrape and Click "View Details" )
https://drivers.uber.com/earnings/activities


Trip Type/ Date and Time
<p class="_css-eJDa-Df">Comfort Electric Tue Dec 31, 2024 18:55</p> 

Earnings
<p class="_css-jYoXid">$16.46</p> Earnings


In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Make sure Chrome was started with:
#   chrome.exe --remote-debugging-port=9222
options = Options()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

driver = webdriver.Chrome(options=options)



xpath = "//p[@class='_css-eJDa-Df'] | //p[@class='_css-jYoXid']"

elements = driver.find_elements(By.XPATH, xpath)
collected_values = [el.get_attribute("textContent") for el in elements]


print(collected_values)






['$16.46', 'Comfort Electric Sun Jan 5, 2025 14:36', '$18.59', 'Comfort Electric Sun Jan 5, 2025 12:55', '$14.81', 'UberX Sat Jan 4, 2025 21:19', '$15.92', 'UberX Sat Jan 4, 2025 20:56', '$20.11', 'UberX Sat Jan 4, 2025 20:26', '$21.53', 'UberX Sat Jan 4, 2025 19:10', '$16.23', 'Comfort Electric Sat Jan 4, 2025 18:48', '$9.50', 'UberX Sat Jan 4, 2025 17:55', '$4.49', 'UberX Sat Jan 4, 2025 17:43', '$7.56', 'UberX Sat Jan 4, 2025 17:11', '$6.81', 'UberX Sat Jan 4, 2025 16:54', '$4.49', 'UberX Sat Jan 4, 2025 16:30', '$29.59', 'UberX Wed Jan 1, 2025 19:56', '$6.78', 'UberX Wed Jan 1, 2025 19:28', '$22.19', 'UberX Wed Jan 1, 2025 19:06', '$0.00', 'UberX Wed Jan 1, 2025 18:52', '$10.40', 'Comfort Electric Wed Jan 1, 2025 18:42', '$10.43', 'UberX Wed Jan 1, 2025 18:03', '$17.29', 'UberX Wed Jan 1, 2025 13:58', '$15.17', 'Comfort Electric Wed Jan 1, 2025 13:26', '$0.00', 'Premier Wed Jan 1, 2025 02:29', '$34.92', 'Premier Wed Jan 1, 2025 02:15', '$50.00', '{0} Trip Quest Wed Jan 1, 2025 02:0

Switched to tab: Uber | Trip Details
Trip Details page loaded successfully.
Dropdown elements found: 0
Error interacting with Trip Details tab: No dropdown elements found.


### Clip Board.for a possible loop, but unable to find the button to clickk on.

In [157]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

def scrape_trip_details(driver):
    """
    Scrapes the details page for a trip.
    Waits until a key element (the 'Fare' button) is clickable, then uses a composite XPath
    to collect various pieces of detail information.
    Returns a list of text values.
    """
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@role='button' and .//p[text()='Fare']]"))
        )
    except Exception as e:
        print("❌ Error waiting for details page to load:", e)
        return []
    
    details_xpath = (
        "//p[@class='_css-eXlLWF']"
        " | //span[@class='_css-jXJAeO']"
        " | //div[contains(@class, '_css-dTqljZ')]"
        " | //div[contains(@class, '_css-cFyYHT')]"
        " | //div[contains(@class, '_css-XloEJ')]"
        " | //p[@data-baseweb='typo-paragraphmedium' and contains(@class, '_css-dTqljZ')]"
    )
    
    elements = driver.find_elements(By.XPATH, details_xpath)
    collected_values = [el.text for el in elements]
    return collected_values

def scrape_first_n_trips_no_refresh(driver, n=5):
    """
    Processes the first n trip rows on the activities page without reloading it.
    For each row:
      - Uses the trip type (from <p data-baseweb="typo-paragraphmedium" class="_css-jGXqXW">)
        and date/time (from <p class="_css-hmIphL">) as unique identifiers.
      - Also extracts the earnings value (from <p data-baseweb="typo-monoparagraphmedium" class="_css-hjwQgK">).
      - Opens the details page in a new tab (via Control+Click), scrapes its details,
        and then combines all the data into one list.
    Returns a list of dictionaries containing the combined data.
    """
    trips_data = []
    main_window = driver.current_window_handle

    # XPath that defines a trip row.
    trip_rows_xpath = (
        "//div[.//button[contains(text(),'View Details')] "
        "and .//p[@data-baseweb='typo-paragraphmedium' and contains(@class, '_css-jGXqXW')] "
        "and .//p[contains(@class, '_css-hmIphL')]]"
    )
    
    # Get all trip rows (more than n if available).
    all_rows = driver.find_elements(By.XPATH, trip_rows_xpath)
    if len(all_rows) < n:
        print(f"Only found {len(all_rows)} rows; expected at least {n}.")
        n = len(all_rows)
    
    # Build a list of unique identifiers for the first n rows.
    # We use the combination: trip type | date/time | earnings.
    identifiers = []
    for row in all_rows[:n]:
        try:
            trip_type = row.find_element(
                By.XPATH, ".//p[@data-baseweb='typo-paragraphmedium' and contains(@class, '_css-jGXqXW')]"
            ).text.strip()
        except Exception as e:
            trip_type = ""
        try:
            date_time = row.find_element(
                By.XPATH, ".//p[contains(@class, '_css-hmIphL')]"
            ).text.strip()
        except Exception as e:
            date_time = ""
        try:
            earnings = row.find_element(
                By.XPATH, ".//p[@data-baseweb='typo-monoparagraphmedium' and contains(@class, '_css-hjwQgK')]"
            ).text.strip()
        except Exception as e:
            earnings = ""
        
        # Combine into an identifier string.
        identifier = f"{trip_type}|{date_time}|{earnings}"
        identifiers.append(identifier)
    
    print(f"Found {len(identifiers)} unique row identifiers to process.")
    
    # Process each row by re-locating it using the unique identifier.
    for index, identifier in enumerate(identifiers):
        # Split the identifier into its components.
        try:
            trip_type_val, date_time_val, earnings_val = identifier.split("|")
        except Exception as e:
            print(f"❌ Error splitting identifier for row {index}: {identifier}")
            continue
        
        # Construct an XPath that locates a row with the matching trip type and date/time.
        # (Earnings might be omitted if not reliable for identification.)
        row_xpath = (
            f"//div[.//p[@data-baseweb='typo-monoparagraphmedium' and contains(@class, '_css-jGXqXW') "
            f"and normalize-space(text())='{trip_type_val}'] "
            f"and .//p[contains(@class, '_css-hmIphL') and normalize-space(text())='{date_time_val}']]"
        )
        try:
            row = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, row_xpath))
            )
        except Exception as e:
            print(f"❌ Could not find row for identifier '{identifier}': {e}")
            continue
        
        print(f"Processing row {index}: Type = '{trip_type_val}', Date/Time = '{date_time_val}', Earnings = '{earnings_val}'")
        
        # Open the details page in a new tab.
        try:
            details_button = row.find_element(By.XPATH, ".//button[contains(text(),'View Details')]")
            actions = ActionChains(driver)
            actions.key_down(Keys.CONTROL).click(details_button).key_up(Keys.CONTROL).perform()
            print(f"✅ Opened details in new tab for row {index}")
        except Exception as e:
            print(f"❌ Error opening details page for row {index}: {e}")
            continue
        
        # Wait for the new tab to appear and switch to it.
        try:
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
        except TimeoutException:
            print(f"❌ Timeout waiting for new tab for row {index}")
            continue
        
        new_handles = driver.window_handles
        details_window = [handle for handle in new_handles if handle != main_window][0]
        driver.switch_to.window(details_window)
        
        # Scrape the details page.
        details = scrape_trip_details(driver)
        print(f"Scraped details for row {index}: {details}")
        
        # Combine summary and details into one list.
        combined_details = [trip_type_val, date_time_val, earnings_val] + details
        
        # Close the details tab and switch back to the main window.
        driver.close()
        driver.switch_to.window(main_window)
        
        trip_data = {"trip_index": index, "combined_details": combined_details}
        trips_data.append(trip_data)
        
        # Short pause for stability.
        time.sleep(2)
    
    return trips_data

# --------------------------
# Example usage:
# (Assume the driver is already attached and on the https://drivers.uber.com/earnings/activities page
# with all trip rows loaded.)
trips = scrape_first_n_trips_no_refresh(driver, n=5)
print("Scraped trips:", trips)


Found 5 unique row identifiers to process.
❌ Could not find row for identifier 'Comfort Electric|Sunday, January 5th, 2025|$16.46': Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6C73102F5+28725]
	(No symbol) [0x00007FF6C7272AE0]
	(No symbol) [0x00007FF6C710510A]
	(No symbol) [0x00007FF6C71593D2]
	(No symbol) [0x00007FF6C71595FC]
	(No symbol) [0x00007FF6C71A3407]
	(No symbol) [0x00007FF6C717FFEF]
	(No symbol) [0x00007FF6C71A0181]
	(No symbol) [0x00007FF6C717FD53]
	(No symbol) [0x00007FF6C714A0E3]
	(No symbol) [0x00007FF6C714B471]
	GetHandleVerifier [0x00007FF6C763F30D+3366989]
	GetHandleVerifier [0x00007FF6C76512F0+3440688]
	GetHandleVerifier [0x00007FF6C76478FD+3401277]
	GetHandleVerifier [0x00007FF6C73DAAAB+858091]
	(No symbol) [0x00007FF6C727E74F]
	(No symbol) [0x00007FF6C727A304]
	(No symbol) [0x00007FF6C727A49D]
	(No symbol) [0x00007FF6C7268B69]
	BaseThreadInitThunk [0x00007FF8A091E8D7+23]
	RtlUserThreadStart [0x00007FF8A289BF2C+44]

❌ Could not find row for identifier 'Comfor